In [1]:
import datetime
import pandas as pd

In [2]:
def getData(domain):
    foursquare = open(f'./input/{domain}_data.txt', 'r').readlines()
    foursquare = [k.split('_') for k in foursquare]
    foursquare=[[id,datetime.datetime.strptime(date, "%Y%m%d%H%M"),eval(lon),eval(lat)] for id,date,lat,lon in foursquare]
    foursquare=pd.DataFrame(foursquare,columns=['userId','date','lon','lat'])
    beginTime=datetime.datetime.strptime('200001010000', "%Y%m%d%H%M")
    foursquare['dateId']=foursquare.date.apply(lambda x:(x-beginTime).days)
    foursquare['binId']=foursquare.apply(lambda x:str(int((round(x[2],2)+180)*1e7+(round(x[3],2)+180)*100)),axis=1)
    foursquare=foursquare[['userId', 'dateId', 'binId']]
    foursquare.sort_values(by=['userId', 'dateId'], ascending=True, inplace=True)
    foursquare.reset_index(drop=True,inplace=True)
    foursquare['domain']=[domain]*len(foursquare)
    return foursquare


In [3]:
def flat2txt(dff):
    df=dff.reset_index(drop=True)
    userid=df.userId[0]
    ans=userid+';|'
    ans+=df[['dateId','binId']].apply(lambda x:str(x[0])+','+str(x[1])+'|',axis=1).sum()
    return ans
def write2txt(df,domain):
    foursquareExample=df.groupby(by='userId').apply(flat2txt).tolist()
    fw = open(f'./input/{domain}_example.txt', 'w')
    for i in foursquareExample:
        fw.write(i+'\n')
    fw.close()

In [4]:
foursquare=getData(domain='fs')
write2txt(foursquare,domain='fs')
twitter=getData(domain='tt')
write2txt(twitter,domain='tt')

In [5]:
data=twitter.append(foursquare).reset_index(drop=True)
data

,userId,dateId,binId,domain
0,1,4235,223120133,tt
1,1,4236,221920130,tt
2,1,4236,221920130,tt
3,1,4238,221420130,tt
4,1,4238,221420130,tt
...,...,...,...,...
98247,99,4373,1053621945,fs
98248,99,4391,1059922071,fs
98249,99,4422,235520072,fs
98250,99,4481,1060222074,fs


$$\rho_l \approx \frac{\sum_u \sum_t a_i(u, l, t)}{\sum_u \sum_t \sum_l a_i(u, l, t)}$$

In [6]:
rhoL=data.groupby(by='binId').apply(lambda x:x.shape[0]/data.shape[0])
rhoL=pd.DataFrame(rhoL,columns=['rhoL'])
rhoL=rhoL.to_dict()['rhoL']

$$p_2 \approx \frac{\sum_u \sum_t \sum_l \min \left(a_1(u, l, t), a_2\left(\sigma_I(u), l, t\right)\right)}{\sum_u \sum_t \sum_l a_1(u, l, t)}$$

$$p_1 \approx \frac{\sum_u \sum_t \sum_l \min \left(a_1(u, l, t), a_2\left(\sigma_I(u), l, t\right)\right)}{\sum_u \sum_t \sum_l a_2(u, l, t)}$$
$$
1=foursquare
$$
$$
2=twitter
$$

In [7]:
numerator=data.groupby(by=['userId', 'dateId', 'binId']).apply(lambda x:min(x[x['domain']=='fs'].shape[0],x[x['domain']=='tt'].shape[0])).sum()

In [8]:
p1=numerator/data[data['domain']=='tt'].shape[0]
p2=numerator/data[data['domain']=='fs'].shape[0]

In [9]:
p1

0.00129366106080207

In [10]:
p2

0.001536235110764778

$$\lambda \approx \frac{1}{(|U|+|V|)|T|} \sum_l\left(\frac{\sum_{u, t} a_1(u, l, t)}{p_1 \rho_l}+\frac{\sum_{v, t} a_2(v, l, t)}{p_2 \rho_l}\right)$$

In [11]:
T=data.dateId.max()-data.dateId.min()
UPlusV=foursquare.userId.unique().shape[0]+twitter.userId.unique().shape[0]

In [12]:
def caculate(df):
    binId=df.binId.tolist()[0]
    rho_l=rhoL[binId]
    return df[df['domain']=='fs'].shape[0]/(p1*rho_l)+df[df['domain']=='tt'].shape[0]/(p2*rho_l)

In [13]:
Lambda=data.groupby(by='binId').apply(caculate).sum()/(UPlusV*T)

In [14]:
Lambda

266419.927836945